<a href="https://colab.research.google.com/github/wtsyang/UserIntentPrediction/blob/BERT/BERT/LSTM-multiPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/UserIntentPrediction'

/content/drive/My Drive/UserIntentPrediction


In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers  import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import text, sequence
from gensim.models import KeyedVectors
from sklearn.model_selection  import train_test_split
import pickle
import sklearn
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import smart_cond
from functools import partial

print('Tensorflow Version:',tf.__version__)

Tensorflow Version: 2.2.0-rc2


In [0]:
NUM_MODELS = 1
BATCH_SIZE = 32
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
EPOCHS = 16
MAX_LEN = 1259
N_CHANNELS=768
N_CLASS=12

## Loading the dataset


In [0]:
Train=pd.read_csv('data/Train_Preprocessing.csv').reset_index(drop=True)
Valid=pd.read_csv('data/Valid_Preprocessing.csv').reset_index(drop=True)
Test=pd.read_csv('data/Test_Preprocessing.csv').reset_index(drop=True)

In [0]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, pdDataFrame, dbName, labels=['oQ', 'RQ', 'CQ', 'FD', 'FQ', 'IR', 'PA', 'PF', 'NF', 'GG', 'JK', 'O'],\
                 batch_size=BATCH_SIZE, dim=MAX_LEN, n_channels=N_CHANNELS,\
                 n_classes=N_CLASS, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = pdDataFrame
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.dbName=dbName
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = self.list_IDs.iloc[indexes,:]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp.reset_index(drop=True))

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim, self.n_channels))
        y = np.zeros((self.batch_size,self.n_classes), dtype=int)

        # Generate data
        for i in range(len(list_IDs_temp)):
            utterenceID=list_IDs_temp.loc[i,'id']
            diaglogID=list_IDs_temp.loc[i,'diaglogID']
            try:
              temp=np.load('BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
              X[i,0:temp.shape[0],:] =temp 
              del temp
            except:
              print('Faile to load the data: BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
            # Store sample
            # Store class
            y[i,:] = np.array(list_IDs_temp.iloc[i,0:12])
        Y=[]
        for i in range(self.n_classes):
          Y+=[y[:,i].reshape((self.batch_size,))]
        return X, Y

In [0]:
classWeight_Dict={}
for i in range(N_CLASS):
  ratioTrue=np.sum(Train.iloc[:,i])/len(Train)
  classWeight_Dict['output'+str(i+1)]={0:1+1/((1-ratioTrue)/(ratioTrue)+1),1:1+(1-ratioTrue)/(ratioTrue)/((1-ratioTrue)/(ratioTrue)+1)}

In [0]:
training_generator = DataGenerator(Train,'Train')
validation_generator = DataGenerator(Valid,'Valid')

In [68]:
classWeight_Dict

{'output1': {0: 1.2345571818407344, 1: 1.7654428181592656},
 'output10': {0: 1.0336144877201687, 1: 1.9663855122798313},
 'output11': {0: 1.0100471347060282, 1: 1.9899528652939718},
 'output12': {0: 1.0018605805011163, 1: 1.9981394194988837},
 'output2': {0: 1.0604068469362442, 1: 1.9395931530637558},
 'output3': {0: 1.074423220044654, 1: 1.925576779955346},
 'output4': {0: 1.2470850905482511, 1: 1.7529149094517489},
 'output5': {0: 1.0875713222525427, 1: 1.912428677747457},
 'output6': {0: 1.1071694368643017, 1: 1.8928305631356983},
 'output7': {0: 1.3979161498387498, 1: 1.6020838501612502},
 'output8': {0: 1.1070453981642272, 1: 1.8929546018357728},
 'output9': {0: 1.076779955346068, 1: 1.923220044653932}}

## Build the model


In [0]:
def binary_crossentropy(y_true, y_pred, weights,from_logits=False,label_smoothing=0):

    y_pred = ops.convert_to_tensor(y_pred)
    y_true = math_ops.cast(y_true, y_pred.dtype)
    label_smoothing = ops.convert_to_tensor(label_smoothing, dtype=K.floatx())
    def _smooth_labels():
      return y_true * (1.0 - label_smoothing) + 0.5 * label_smoothing
    y_true = smart_cond.smart_cond(label_smoothing,_smooth_labels, lambda: y_true)
    
    mask0 = tf.subtract(tf.constant(1.0, dtype=K.floatx()),y_true)
    mask0=tf.math.scalar_mul(tf.constant(weights[0], dtype=K.floatx()),mask0)
    mask1 =tf.math.scalar_mul(tf.constant(weights[1], dtype=K.floatx()),y_true)
    mask=tf.math.add(mask0,mask1)

    return K.mean(tf.math.multiply(K.binary_crossentropy(y_true, y_pred, from_logits=from_logits),mask), axis=-1)

In [0]:
def build_model():
    inputs = Input(shape=(MAX_LEN,N_CHANNELS))
    x = SpatialDropout1D(0.2)(inputs)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    RESULT=[]
    for i in range(N_CLASS):
      RESULT+=[Dense(1, activation='sigmoid',name='output'+str(i+1))(hidden)]
    LOSS={}
    for i in  range(N_CLASS):
      LOSS['output'+str(i+1)]=partial(binary_crossentropy, weights=classWeight_Dict['output'+str(i+1)])
      LOSS['output'+str(i+1)].__name__ = 'loss'+str(i+1)

    model = Model(inputs=inputs, outputs=RESULT)
    model.compile(loss=LOSS, optimizer='adam',metrics=[tf.keras.metrics.binary_accuracy])

    return model

In [70]:
model = build_model()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1259, 768)]  0                                            
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 1259, 768)    0           input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 1259, 256)    918528      spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 1259, 256)    394240      bidirectional_4[0][0]            
____________________________________________________________________________________________

In [0]:
callback=ReduceLROnPlateau(patience=1,min_lr=0.00001)
checkpointer = ModelCheckpoint(filepath='BERT/LSTM.h5', verbose=1, save_best_only=True)

In [71]:
model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpointer,callback])

Epoch 1/16
251/251 [==============================] - ETA: 0s - loss: 4.5476 - output1_loss: 0.5219 - output2_loss: 0.3284 - output3_loss: 0.4191 - output4_loss: 0.8164 - output5_loss: 0.4392 - output6_loss: 0.4548 - output7_loss: 0.5384 - output8_loss: 0.3899 - output9_loss: 0.3849 - output10_loss: 0.1403 - output11_loss: 0.0860 - output12_loss: 0.0282 - output1_binary_accuracy: 0.8291 - output2_binary_accuracy: 0.9366 - output3_binary_accuracy: 0.9212 - output4_binary_accuracy: 0.7328 - output5_binary_accuracy: 0.9106 - output6_binary_accuracy: 0.8926 - output7_binary_accuracy: 0.8474 - output8_binary_accuracy: 0.9035 - output9_binary_accuracy: 0.9201 - output10_binary_accuracy: 0.9653 - output11_binary_accuracy: 0.9898 - output12_binary_accuracy: 0.9941 
Epoch 00001: val_loss improved from inf to 3.62126, saving model to BERT/LSTM.h5
251/251 [==============================] - 6120s 24s/step - loss: 4.5476 - output1_loss: 0.5219 - output2_loss: 0.3284 - output3_loss: 0.4191 - output4

## Prediction




In [0]:
class testDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, pdDataFrame, dbName, labels=['oQ', 'RQ', 'CQ', 'FD', 'FQ', 'IR', 'PA', 'PF', 'NF', 'GG', 'JK', 'O'],\
                 batch_size=25, dim=MAX_LEN, n_channels=N_CHANNELS,\
                 n_classes=N_CLASS, shuffle=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = pdDataFrame
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.dbName=dbName
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = self.list_IDs.iloc[indexes,:]

        # Generate data
        X = self.__data_generation(list_IDs_temp.reset_index(drop=True))

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim, self.n_channels))
        #y = np.zeros((self.batch_size,self.n_classes), dtype=int)

        # Generate data
        for i in range(len(list_IDs_temp)):
            utterenceID=list_IDs_temp.loc[i,'id']
            diaglogID=list_IDs_temp.loc[i,'diaglogID']
            try:
              temp=np.load('BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
              X[i,0:temp.shape[0],:] =temp 
              del temp
            except:
              print('Faile to load the data: BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
            # Store sample
            # Store class
            #y[i,:] = np.array(list_IDs_temp.iloc[i,0:12])

        return X

In [0]:
test_generator=testDataGenerator(Test,'Test')

In [74]:
prediction = model.predict_generator(test_generator)

Instructions for updating:
Please use Model.predict, which supports generators.


In [0]:
Prediction=np.array(prediction)

In [76]:
IOU=0
for i in range(len(Test)):
  pred=Prediction[0:N_CLASS,i,0]
  labels=np.array(Test.iloc[i,0:N_CLASS])
  ioU=np.sum((pred>=0.5)&(labels==1))/np.sum( (pred>=0.5) | (labels==1))
  IOU+=ioU
IOU=IOU/len(Test)
print(IOU)

0.640108108108108


In [77]:
ACC=0
for i in range(len(Test)):
  pred=Prediction[0:N_CLASS,i,0]
  labels=np.array(Test.iloc[i,0:N_CLASS])
  acc=np.sum((pred>=0.5)==labels)/12
  ACC+=acc
ACC=ACC/len(Test)
print(ACC)

0.9233333333333331
